In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup
import re

In [3]:
def get_nutritional_values(soup):
    nutritional_values = {}
    nutrient_gauges = soup.find_all("div", class_="new-product-page__nutrient-gauge")
    
    for gauge in nutrient_gauges:
        title = gauge.find("div", class_="new-product-page__nutrient-gauge-title").text
        value = gauge.find("div", class_="new-product-page__nutrient-gauge-text").text
        nutritional_values[title] = value

    keys = {
        'calories': "Wartość energetyczna (kcal)",
        'fat': "Tłuszcz (g)",
        'carbs': "Węglowodany (g)",
        'sugar': "w tym cukry (g)",
        'protein': "Białko (g)"
    }
    
    nutritional_values_converted = {k: float(nutritional_values[v].split()[0]) for k, v in keys.items()}
    return nutritional_values_converted

In [4]:
def get_price(soup):
    price_div = soup.find("div", class_="new-product-page__prices_price")
    price_num = price_div.find("span", class_="price_num").text
    price_decimals = price_div.find("span", class_="price_decimals").text
    price = float(f"{price_num}.{price_decimals}")
    return price

In [5]:
def get_portion_size(soup):
    product_div = soup.find('div', class_="product-grammage")
    return float(product_div.text.split('\xa0')[0])

In [6]:
def get_frisco_product_data(api_response):
    product_url = api_response['productUrl']
    html = requests.get(product_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    data = get_nutritional_values(soup)
    data['package_price'] = get_price(soup)
    data['package_weight'] = get_portion_size(soup)
    data['type'] = api_response['name']['pl']
    data['name'] = api_response['brand'] + ((' ' + api_response['subbrand']) if 'subbrand' in api_response else '')
    return data

In [7]:
frisco_ids = [145426, 145237, 125131, 6073]

In [8]:
responses = []
for product_id in frisco_ids:
    response = requests.get(f"https://www.frisco.pl/app/commerce/api/v1/offer/products/{product_id}").json()
    responses.append(response)

In [9]:
[
    get_frisco_product_data(api_response) for api_response in responses
]

[{'calories': 359.0,
  'fat': 7.0,
  'carbs': 54.0,
  'sugar': 1.1,
  'protein': 14.0,
  'package_price': 4.69,
  'package_weight': 500.0,
  'type': 'Płatki owsiane górskie extra',
  'name': 'SANTE'},
 {'calories': 64.0,
  'fat': 0.0,
  'carbs': 4.1,
  'sugar': 4.1,
  'protein': 12.0,
  'package_price': 7.89,
  'package_weight': 450.0,
  'type': 'Jogurt typu islandzkiego naturalny',
  'name': 'PIĄTNICA Skyr'},
 {'calories': 603.0,
  'fat': 48.0,
  'carbs': 12.0,
  'sugar': 6.7,
  'protein': 27.0,
  'package_price': 14.45,
  'package_weight': 500.0,
  'type': 'Pasta orzechowa crunchy',
  'name': 'SANTE GO ON!'},
 {'calories': 828.0,
  'fat': 92.0,
  'carbs': 0.0,
  'sugar': 0.0,
  'protein': 0.0,
  'package_price': 41.99,
  'package_weight': 500.0,
  'type': 'Oliwa z oliwek extra vergine',
  'name': 'MONINI GranFruttato'}]